In [ ]:
import torch
import time

# Define the neural network model
class FullyConnectedNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(FullyConnectedNN, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.layers = torch.nn.ModuleList()
        self.layers.append(torch.nn.Linear(input_dim, hidden_dim))
        for _ in range(num_layers - 1):
            self.layers.append(torch.nn.Linear(hidden_dim, hidden_dim))
    
    def forward(self, x):
        for layer in self.layers:
            x = torch.tanh(layer(x))
        return x

# # Define the function u_theta
# def u_theta(x, model):
#     return model(x)

# Define a function to compute the time for u_theta, nabla u_theta, and Delta u_theta
def compute_times(model, input_dim, num_samples=1000):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    u_theta_time       = 0
    nabla_u_theta_time = 0
    delta_u_theta_time = 0
    for step in range(2000):
        # Randomly generate sample points
        x = torch.randn(num_samples, input_dim).to(device).requires_grad_(True)
        
        # Compute u_theta and measure the time
        start_time = time.time()
        u = model(x)
        if step >= 1000:
            u_theta_time += time.time() - start_time

        # Compute nabla u_theta and measure the time
        start_time = time.time()
        du = torch.autograd.grad(u, x, 
                                grad_outputs=torch.ones_like(u), 
                                create_graph=True, 
                                retain_graph=True)[0]
        if step >= 1000:
            nabla_u_theta_time += time.time() - start_time

        # Compute Delta u_theta and measure the time
        start_time = time.time()
        laplace = torch.zeros_like(u)
        for i in range(input_dim):
            d2u = torch.autograd.grad(du[:, i], x, 
                                    grad_outputs=torch.ones_like(du[:, i]), 
                                    create_graph=True, 
                                    retain_graph=True)[0][:, i]
            laplace += d2u.reshape(-1, 1)
        if step >= 1000:
            delta_u_theta_time += time.time() - start_time

    return u_theta_time, nabla_u_theta_time, delta_u_theta_time

# Loop through different input dimensions and compute the times
for input_dim in [1, 2, 10, 50, 100, 200]:
    print("============= input_dim", input_dim, "==============")
    # Parameter settings
    hidden_dim = 200
    num_layers = 6

    # Create the neural network model
    model = FullyConnectedNN(input_dim, hidden_dim, num_layers)

    # Compute the times
    u_theta_time, nabla_u_theta_time, delta_u_theta_time = compute_times(model, input_dim)

    print("Computing time for u_theta:", u_theta_time)
    print("Computing time for nabla u_theta:", nabla_u_theta_time)
    print("Computing time for Delta u_theta:", delta_u_theta_time)